In [ ]:
!pip install gradio openai

import gradio as gr
from openai import OpenAI
import sqlite3

# Load your API key securely from Colab secrets
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

# Initialize SQLite database
conn = sqlite3.connect("todo.db", check_same_thread=False)
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS tasks (id INTEGER PRIMARY KEY AUTOINCREMENT, task TEXT)")
conn.commit()

# CRUD operations
def add_task(task):
    if task.strip():
        cursor.execute("INSERT INTO tasks (task) VALUES (?)", (task,))
        conn.commit()
    return read_tasks()

def read_tasks():
    cursor.execute("SELECT id, task FROM tasks")
    rows = cursor.fetchall()
    return [{"id": row[0], "task": row[1]} for row in rows]


def update_task(index, new_task):
    try:
        cursor.execute("UPDATE tasks SET task = ? WHERE id = ?", (new_task, int(index)))
        conn.commit()
    except:
        pass
    return read_tasks()

def delete_task(index):
    try:
        task_id = int(index)  # ensure integer
        cursor.execute("DELETE FROM tasks WHERE id = ?", (task_id,))
        conn.commit()
    except Exception as e:
        print("Delete error:", e)
    return read_tasks()


# Bonus: AI Summarization of all tasks
def summarize_tasks():
    cursor.execute("SELECT task FROM tasks")
    tasks = [row[0] for row in cursor.fetchall()]
    if not tasks:
        return "No tasks available to summarize."
    text = "\n".join(tasks)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": f"Summarize these tasks in a concise way:\n{text}"}]
        )
        summary = response.choices[0].message.content
        return summary
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio UI
def interface():
    with gr.Blocks() as demo:
        gr.Markdown("## 📝 To-Do Manager with AI Summarizer (SQLite-backed)")

        with gr.Row():
            task_input = gr.Textbox(label="New Task")
            add_btn = gr.Button("Add Task")

        tasks_output = gr.JSON(label="Current Tasks (id: task)")

        with gr.Row():
            idx_update = gr.Number(label="Task ID to Update", precision=0)
            new_task = gr.Textbox(label="Updated Task")
            update_btn = gr.Button("Update Task")

        with gr.Row():
            idx_delete = gr.Number(label="Task ID to Delete", precision=0)
            delete_btn = gr.Button("Delete Task")

        summarize_btn = gr.Button("Summarize Tasks with AI")
        summary_output = gr.Textbox(label="AI Summary")

        # Button actions
        add_btn.click(add_task, inputs=task_input, outputs=tasks_output)
        update_btn.click(update_task, inputs=[idx_update, new_task], outputs=tasks_output)
        delete_btn.click(delete_task, inputs=idx_delete, outputs=tasks_output)
        summarize_btn.click(summarize_tasks, outputs=summary_output)

        # Show tasks initially
        demo.load(read_tasks, outputs=tasks_output)

    return demo

app = interface()

if __name__ == "__main__":
    app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd5f9dd5a99a9de368.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
